In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict

In [2]:
homeDir= '/Users/humnaawan'

# database from the new afterburner: has random rotDithers in ditheredRotTelPos to compare the stacker output.
dbfile = '%s/lsstData/minion_1016_sqlite_new_dithers.db'%homeDir

Test the new stacker on the opsim database; add the rotDither column.

In [3]:
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as mafUtils
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
from mafContrib import RandomRotDitherPerFilterChangeStacker

In [4]:
# set up to run the stacker.
opsdb = db.OpsimDatabase(dbfile)  #database
metric = metrics.PassMetric()  # want to access the database; no analysis needed
slicer = slicers.OneDSlicer(sliceColName='night', binsize=1)   # essentially accessing all nights
sqlconstraint= None #'night<50'
stackerList = [RandomRotDitherPerFilterChangeStacker(randomSeed=42)]  # seed as in the OpSim afterburner; for testing here

In [5]:
outDir = '%s/stackers_test'%homeDir
resultsDb = db.ResultsDb(outDir=outDir)

# set up metric bundle
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint= sqlconstraint, stackerList=stackerList)
bgroup = metricBundles.MetricBundleGroup({'check': bundle}, opsdb, outDir=outDir,
                                         resultsDb=resultsDb, saveEarly= False)
# run the bundle
bgroup.runAll()

Querying database with no constraint.
Found 2447931 visits
Running:  ['check']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [6]:
# see what columns are available
print(bgroup.simData.dtype.names)

('obsHistID', 'rotTelPos', 'night', 'filter', 'randomDitherPerFilterChangeRotTelPos')


Great - the new column is added.

To check things, now read in the afterburner-output data manually; the db file has ditheredRotTelPos which was added by the afterburner. The column added by the stacker should match the one added by the afterburner.

In [7]:
# read in the data
colnames = ['obsHistID', 'filter', 'rotTelPos', 'ditheredRotTelPos']
simdata = opsdb.fetchMetricData(colnames, sqlconstraint)

In [8]:
# to compare with the dithers added by the stacker used here, need to match obsIDs.
if not np.array_equal(bgroup.simData['obsHistID'], simdata['obsHistID']):
    print('Finding inds to match obsIDs from the two simData arrays.')
    inds= []  # needed for subsample checks
    for obsID in simdata['obsHistID']:
        inds.append(np.where(bgroup.simData['obsHistID']==obsID)[0][0])
else: 
    inds= None

In [9]:
# compare all entries to ensure the newly added column (randomDitherPerFilterChangeRotTelPos) matches with 'ditheredRotTelPos
print('Checking a total of %s observations.'%len(simdata['obsHistID']))

err_tol= 0.0001
if inds is None:
    diff= abs(bgroup.simData['randomDitherPerFilterChangeRotTelPos']-simdata['ditheredRotTelPos'])
else:
    diff= abs(bgroup.simData['randomDitherPerFilterChangeRotTelPos'][inds]-simdata['ditheredRotTelPos'][inds])
    
if np.any(diff>err_tol):
    # ok something's wrong. break.
    print('Omg')
    ind= np.where(diff>err_tol)[0]
    raise ValueError('Problem: ditheredTel positions dont match: \n %s \n vs.\n Afterburner: %s'%(
                                    bgroup.simData['randomDitherPerFilterChangeRotTelPos'][ind],
                                            simdata['ditheredRotTelPos'][ind]))
print('All good.')

Checking a total of 2447931 observations.
All good.


In [10]:
# print out some entries to see things.
checkList= colnames
indMin, indMax= 250, 300

# to find the dithers added by the stacker used here, need to match obsIDs.
obsIDs= simdata['obsHistID'][indMin:indMax]
inds= []
for obsID in obsIDs:
    inds.append(np.where(bgroup.simData['obsHistID']==obsID)[0][0])
         
# set up a dict of the columns to print. add in order.
simList= OrderedDict()
for col in ['obsHistID', 'filter', 'rotTelPos']:
    simList[col]= simdata[col][indMin:indMax]
simList['rotTelPos_usedByStacker']= bgroup.simData['rotTelPos'][inds]
simList['ditheredRotTelPos']= simdata['ditheredRotTelPos'][indMin:indMax]
simList['ditheredRotTelPos_fromStacker']= bgroup.simData['randomDitherPerFilterChangeRotTelPos'][inds]

# add dithers themselve
simList['rotDither_fromAfterburner']= simdata['ditheredRotTelPos'][indMin:indMax]-simdata['rotTelPos'][indMin:indMax]
simList['rotDither_fromStacker']= simList['ditheredRotTelPos_fromStacker']-simList['rotTelPos_usedByStacker']

pd.DataFrame(simList)

,obsHistID,filter,rotTelPos,rotTelPos_usedByStacker,ditheredRotTelPos,ditheredRotTelPos_fromStacker,rotDither_fromAfterburner,rotDither_fromStacker
0,251,g,0.484243,0.484243,1.900204,1.900204,1.415961,1.415961
1,252,g,0.484404,0.484404,1.900365,1.900365,1.415961,1.415961
2,253,g,0.484569,0.484569,1.900530,1.900530,1.415961,1.415961
3,254,g,0.484740,0.484740,1.900701,1.900701,1.415961,1.415961
4,255,g,0.484915,0.484915,1.900876,1.900876,1.415961,1.415961
5,256,g,0.485096,0.485096,1.901057,1.901057,1.415961,1.415961
6,257,g,0.485281,0.485281,1.901242,1.901242,1.415961,1.415961
7,258,g,0.485472,0.485472,1.901433,1.901433,1.415961,1.415961
8,259,g,0.485666,0.485666,1.901627,1.901627,1.415961,1.415961
9,260,i,0.000000,0.000000,0.728830,0.728830,0.728830,0.728830


Ok great, there's a new rotational dither after a filter change + things match.